# **Importing** **Libraries**

In [ ]:
#Importing all required libraries and packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sb
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from statsmodels.graphics import tsaplots
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
import math
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
from sklearn import model_selection
from math import sqrt
from sklearn.preprocessing import OneHotEncoder
print('Libraries imported.')

Libraries imported.


# **Data Preperation, Pre-Processing, and FE**

## Read Data and take a look

In [ ]:
#Read Pollutants Data for Piccadilly
PollutantsPicc = pd.read_csv('Piccadilly.csv')
PollutantsPicc.head()

In [ ]:
#Read Traffic Data From Piccadilly
traffic22 = pd.read_csv('pvr_2016-03-04_1765d(1).csv')
#Rename date column to match the date column of the Pollutants Data from Piccadilly
traffic22 = traffic22.rename(columns = {"Sdate":"date"})
#Rename Cosit to match the same style as well
traffic22['Cosit'].replace('="MAC030001146"', 'MAC030001146', inplace=True)
#Remove Channel 1 as it recorded no Traffic volume
traffic22 = traffic22[traffic22.LaneDescription != 'Channel 2']
traffic22.head()

In [ ]:
#Read Data from Sharston
PollutantsShar = pd.read_csv('Sharston.csv')
PollutantsShar.head()

In [ ]:
#Read traffic data for Sharston 
traffic3 = pd.read_csv('pvr_2015-06-01_2615d.csv')
#Rename date column to match pollutants data
traffic3 = traffic3.rename(columns = {"Sdate":"date"})
#Remove Channel 1 as it recorded no Traffic volume values
traffic3 = traffic3[traffic3.LaneDescription != 'Channel 2']
traffic3.head()

## Data Pre-processing and Feature Engineering Piccadilly - done

merged data

In [ ]:
#merge traffic data and pollutants data on the date column
df = pd.merge(traffic22,PollutantsPicc[['date','NO2','wd','ws','temp','longitude', 'latitude']],on='date', how='left')
#drop irrelevant variables
df = df.drop(['AvgSpeed', 'PmlHGV', 'Flag Text'], axis = 1)

In [ ]:
#Taking a look at the first 5 rows to make sure merging was effictive
df.head()

In [ ]:
#check for null values
df.isnull().mean()

In [ ]:
#drop null values
df = df.dropna()

In [ ]:
#check correlation between variables
df[['NO2','Volume','ws','wd','temp']].corr()

In [ ]:
#summary statistics for some variables
df[['NO2','Volume','ws','wd','temp']].describe()

In [ ]:
#check variable info including column types
df.info()

In [ ]:
#extract seasonality data drom data column
df['date'] =  pd.to_datetime(df['date'])
df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Hour'] = df['date'].dt.hour
df['DayofWeek'] = df['date'].dt.dayofweek
df['dayofyear'] = df['date'].dt.dayofyear
df['weekofyear'] = df['date'].dt.weekofyear
df['quarter'] = df['date'].dt.quarter

In [ ]:
#create new variables using rolling means function (may not use all)
df['NO2_moving_avg'] = df['NO2'].rolling(window=25).mean()
df['NO2_moving_avg2'] = df['NO2'].rolling(window=100).mean()
df['NO2_moving_avg3'] = df['NO2'].rolling(window=50).mean()
df['NO2_moving_avg4'] = df['NO2'].rolling(window=3).mean()
df['NO2_moving_avg5'] = df['NO2'].rolling(window=30).mean()

In [ ]:
#drop any new missing values generating from moving average variables
df = df.dropna()

In [ ]:
#Transform wind direction from degrees to textual direction (for EDA)
df['wd'] = df['wd'].mask(df['wd'] == 0, 0)
df['wd'] = df['wd'].mask((df['wd'] > 0) & (df['wd'] < 90), 0.5)
df['wd'] = df['wd'].mask(df['wd'] == 90, 1)
df['wd'] = df['wd'].mask((df['wd'] > 90) & (df['wd'] < 180), 1.5)
df['wd'] = df['wd'].mask(df['wd'] == 180, 2)
df['wd'] = df['wd'].mask((df['wd'] > 180) & (df['wd'] < 270), 2.5)
df['wd'] = df['wd'].mask(df['wd'] == 270, 3)
df['wd'] = df['wd'].mask((df['wd'] > 270) & (df['wd'] < 360), 3.5)
df['wd'] = df['wd'].mask(df['wd'] == 360, 'North')

df['wd'] = df['wd'].mask(df['wd'] == 0, 'North')
df['wd'] = df['wd'].mask(df['wd'] == 0.5, 'North East')
df['wd'] = df['wd'].mask(df['wd'] == 1, 'East')
df['wd'] = df['wd'].mask(df['wd'] == 1.5, 'South East')
df['wd'] = df['wd'].mask(df['wd'] == 2, 'South')
df['wd'] = df['wd'].mask(df['wd'] == 2.5, 'South West')
df['wd'] = df['wd'].mask(df['wd'] == 3, 'West')
df['wd'] = df['wd'].mask(df['wd'] == 3.5, 'North West')

In [ ]:
#One hot encode Wind direction
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'wd' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df[['wd1']]).toarray())

#merge one-hot encoded columns back with original DataFrame
df = df.join(encoder_df)

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'Day of week' column 
df[['ohe1','ohe2','ohe3','oh4','o5','o6','o7']] = encoder.fit_transform(df[['DayofWeek']]).toarray()

Pollutants data

In [ ]:
#drop irrelevant variables
PollutantsPicc = PollutantsPicc.drop(['NOXasNO2', 'SO2', 'NV2.5',
                                      'V2.5','AT2.5','AP2.5','AT25','AP25',
                                      'PM10','RAWPM25'], axis = 1)

In [ ]:
#check for missing values
PollutantsPicc.isnull().mean()

In [ ]:
#correlation for data
PollutantsPicc[['NO2', 'ws','wd','temp']].corr()

In [ ]:
#Summary statistics for data
PollutantsPicc[['NO2', 'ws','wd','temp']].describe()

In [ ]:
#Transform wind direction from degrees to textual direction (for EDA)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 0, 0)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask((PollutantsPicc['wd'] > 0) & (PollutantsPicc['wd'] < 90), 0.5)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 90, 1)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask((PollutantsPicc['wd'] > 90) & (PollutantsPicc['wd'] < 180), 1.5)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 180, 2)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask((PollutantsPicc['wd'] > 180) & (PollutantsPicc['wd'] < 270), 2.5)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 270, 3)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask((PollutantsPicc['wd'] > 270) & (PollutantsPicc['wd'] < 360), 3.5)
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 360, 'North')

PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 0, 'North')
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 0.5, 'North East')
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 1, 'East')
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 1.5, 'South East')
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 2, 'South')
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 2.5, 'South West')
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 3, 'West')
PollutantsPicc['wd'] = PollutantsPicc['wd'].mask(PollutantsPicc['wd'] == 3.5, 'North West')

In [ ]:
#One hot encode Wind direction
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'wd' column 
encoder_df = pd.DataFrame(encoder.fit_transform(PollutantsPicc[['wd']]).toarray())

#merge one-hot encoded columns back with original DataFrame
PollutantsPicc = PollutantsPicc.join(encoder_df)

In [ ]:
#check variable info including column types
PollutantsPicc.info()

In [ ]:
#extract new features from date column
PollutantsPicc['date'] =  pd.to_datetime(PollutantsPicc['date'])
PollutantsPicc['Year'] = PollutantsPicc['date'].dt.year
PollutantsPicc['Month'] = PollutantsPicc['date'].dt.month
PollutantsPicc['Hour'] = PollutantsPicc['date'].dt.hour
PollutantsPicc['DayofWeek'] = PollutantsPicc['date'].dt.dayofweek
PollutantsPicc['dayofyear'] = PollutantsPicc['date'].dt.dayofyear
PollutantsPicc['weekofyear'] = PollutantsPicc['date'].dt.weekofyear
PollutantsPicc['quarter'] = PollutantsPicc['date'].dt.quarter
#create new variables using rolling means
PollutantsPicc['NO2_moving_avg'] = PollutantsPicc['NO2'].rolling(window=3).mean()#3hours
PollutantsPicc['NO2_moving_avg2'] = PollutantsPicc['NO2'].rolling(window=25).mean()#24hours
PollutantsPicc['NO2_moving_avg3'] = PollutantsPicc['NO2'].rolling(window=50).mean()#~2days
PollutantsPicc['NO2_moving_avg4'] = PollutantsPicc['NO2'].rolling(window=100).mean()#~4days
PollutantsPicc['NO2_moving_avg5'] = PollutantsPicc['NO2'].rolling(window=200).mean()#~8days

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'dayofweek' column 
PollutantsPicc[['ohe1','ohe2','ohe3','oh4','o5','o6','o7']] = encoder.fit_transform(PollutantsPicc[['DayofWeek']]).toarray()

#merge one-hot encoded columns back with original DataFrame
#PollutantsPicc = PollutantsPicc.join(encoder_df)

In [ ]:
#drop missing values generated from moving average
PollutantsPicc = PollutantsPicc.dropna()

## Data Pre-processing and Feature Engineering Sharston

In [ ]:
#remove empty channel 1 data and change the date format to match that of the pollutants data
traffic3 = traffic3[traffic3.LaneDescription != 'Channel 2']
date_sr = pd.to_datetime(pd.Series(traffic3.date))
traffic3.date = change_format = date_sr.dt.strftime('%d/%m/%Y %H:%M')
 
# Print the formatted date
print(change_format)

merged data

In [ ]:
#Megre the data on date
df2 = pd.merge(traffic3,PollutantsShar[['date','NO2','wd','ws','temp','longitude', 'latitude']],
               on='date', how='left')

In [ ]:
#Drop irrelevant null filled variables
df2 = df2.drop(['AvgSpeed', 'PmlHGV', 'Flag Text'], axis = 1)

In [ ]:
#take a look at the first 5 rows to make sure merging was successful
df2.head()

In [ ]:
#Check for missing values
df2.isnull().mean()

In [ ]:
#drop missing values
df2 = df2.dropna()

In [ ]:
#check correlation between variables
df2.corr()

In [ ]:
#Statistics summary of data
df2[['NO2','Volume','ws','wd','temp']].describe()

In [ ]:
#check variable info including column types
df2.info()

In [ ]:
#New seasonality features extracted
df2['date'] =  pd.to_datetime(df2['date'])
df2['Year'] = df2['date'].dt.year
df2['Month'] = df2['date'].dt.month
df2['Hour'] = df2['date'].dt.hour
df2['DayofWeek'] = df2['date'].dt.dayofweek
df2['dayofyear'] = df2['date'].dt.dayofyear
df2['weekofyear'] = df2['date'].dt.weekofyear
df2['quarter'] = df2['date'].dt.quarter
#rolling means features
df2['NO2_moving_avg'] = df2['NO2'].rolling(window=3).mean()
df2['NO2_moving_avg2'] = df2['NO2'].rolling(window=25).mean()
df2['NO2_moving_avg3'] = df2['NO2'].rolling(window=50).mean()
df2['NO2_moving_avg4'] = df2['NO2'].rolling(window=100).mean()

In [ ]:
#Wind direction from degrees to textual (for EDA)
df2['wd'] = df2['wd'].mask(df2['wd'] == 0.1, 0)
df2['wd'] = df2['wd'].mask((df2['wd'] > 0.1) & (df2['wd'] < 90), 0.5)
df2['wd'] = df2['wd'].mask(df2['wd'] == 90, 1)
df2['wd'] = df2['wd'].mask((df2['wd'] > 90) & (df2['wd'] < 180), 1.5)
df2['wd'] = df2['wd'].mask(df2['wd'] == 180, 2)
df2['wd'] = df2['wd'].mask((df2['wd'] > 180) & (df2['wd'] < 270), 2.5)
df2['wd'] = df2['wd'].mask(df2['wd'] == 270, 3)
df2['wd'] = df2['wd'].mask((df2['wd'] > 270) & (df2['wd'] < 359.9), 3.5)
df2['wd'] = df2['wd'].mask(df2['wd'] == 359.9, 'North')

df2['wd'] = df2['wd'].mask(df2['wd'] == 0, 'North')
df2['wd'] = df2['wd'].mask(df2['wd'] == 0.5, 'North East')
df2['wd'] = df2['wd'].mask(df2['wd'] == 1, 'East')
df2['wd'] = df2['wd'].mask(df2['wd'] == 1.5, 'South East')
df2['wd'] = df2['wd'].mask(df2['wd'] == 2, 'South')
df2['wd'] = df2['wd'].mask(df2['wd'] == 2.5, 'South West')
df2['wd'] = df2['wd'].mask(df2['wd'] == 3, 'West')
df2['wd'] = df2['wd'].mask(df2['wd'] == 3.5, 'North West')

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'wd' column 
encoder_df = pd.DataFrame(encoder.fit_transform(df2[['wd']]).toarray())

#merge one-hot encoded columns back with original DataFrame
df2 = df2.join(encoder_df)

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'dayofweek' column and add them to new columns
df2[['ohe1','ohe2','ohe3','oh4','o5','o6','o7']] = encoder.fit_transform(df2[['DayofWeek']]).toarray()

In [ ]:
#check for missing values
PollutantsShar.isnull().mean()

In [ ]:
#drop missing values
PollutantsShar = PollutantsShar.dropna()

In [ ]:
PollutantsShar.corr()

In [ ]:
#Statistics summary of data
PollutantsShar[['NO2','ws','wd','temp']].describe()

In [ ]:
PollutantsShar.info()

In [ ]:
#extract new seasonality features
PollutantsShar['date'] =  pd.to_datetime(PollutantsShar['date'])
PollutantsShar['Year'] = PollutantsShar['date'].dt.year
PollutantsShar['Month'] = PollutantsShar['date'].dt.month
PollutantsShar['Hour'] = PollutantsShar['date'].dt.hour
PollutantsShar['DayofWeek'] = PollutantsShar['date'].dt.dayofweek
PollutantsShar['dayofyear'] = PollutantsShar['date'].dt.dayofyear
PollutantsShar['weekofyear'] = PollutantsShar['date'].dt.weekofyear
PollutantsShar['quarter'] = PollutantsShar['date'].dt.quarter
PollutantsShar['Day'] = PollutantsShar['date'].dt.day
#new rollings means variables
PollutantsShar['NO2_moving_avg'] = PollutantsShar['NO2'].rolling(window=3).mean()
PollutantsShar['NO2_moving_avg2'] = PollutantsShar['NO2'].rolling(window=25).mean()
PollutantsShar['NO2_moving_avg3'] = PollutantsShar['NO2'].rolling(window=50).mean()
PollutantsShar['NO2_moving_avg4'] = PollutantsShar['NO2'].rolling(window=100).mean()

In [ ]:
#wind direction from degrees to textual
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 0, 0)
PollutantsShar['wd'] = PollutantsShar['wd'].mask((PollutantsShar['wd'] > 0) & (PollutantsShar['wd'] < 90), 0.5)
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 90, 1)
PollutantsShar['wd'] = PollutantsShar['wd'].mask((PollutantsShar['wd'] > 90) & (PollutantsShar['wd'] < 180), 1.5)
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 180, 2)
PollutantsShar['wd'] = PollutantsShar['wd'].mask((PollutantsShar['wd'] > 180) & (PollutantsShar['wd'] < 270), 2.5)
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 270, 3)
PollutantsShar['wd'] = PollutantsShar['wd'].mask((PollutantsShar['wd'] > 270) & (PollutantsShar['wd'] < 360), 3.5)
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 360, 'North')

PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 0, 'North')
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 0.5, 'North East')
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 1, 'East')
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 1.5, 'South East')
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 2, 'South')
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 2.5, 'South West')
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 3, 'West')
PollutantsShar['wd'] = PollutantsShar['wd'].mask(PollutantsShar['wd'] == 3.5, 'North West')

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'wd' column 
encoder_df = pd.DataFrame(encoder.fit_transform(PollutantsShar[['wd']]).toarray())

#merge one-hot encoded columns back with original DataFrame
PollutantsShar = PollutantsShar.join(encoder_df)

In [ ]:
#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'dayofweek' column 
PollutantsShar[['ohe1','ohe2','ohe3','oh4','o5','o6','o7']] = encoder.fit_transform(PollutantsShar[['DayofWeek']]).toarray()

# **Picc&Shar** **EDA**

## EDA

In [ ]:
fig= plt.figure(figsize=(10,5))
PollutantsPicc['NO2'].plot(figsize=(10,6))
PollutantsPicc['NO2'].rolling(window=200).mean().plot()
plt.title('Piccadilly')
plt.ylabel('NO2')

In [ ]:
fig= plt.figure(figsize=(10,5))
PollutantsShar['NO2'].plot(figsize=(10,6))
PollutantsShar['NO2'].rolling(window=200).mean().plot()
plt.ylabel('NO2')
plt.title('Sharston')

In [ ]:
from windrose import WindroseAxes
from matplotlib import pyplot as plt
import matplotlib.cm as cm
import numpy as np
ax = WindroseAxes.from_ax()
ax.bar(PollutantsShar.wd, PollutantsShar.ws, normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
ax.legend(loc='upper left')
ax.set_title('Sharston', loc='right')
ax = WindroseAxes.from_ax()
ax.bar(PollutantsPicc.wd, PollutantsPicc.ws, normed=True, opening=0.8, edgecolor='white')
ax.set_legend()
ax.legend(loc='upper left')
ax.set_title('Piccadilly', loc='right')

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12,6))
sb.kdeplot(PollutantsPicc.NO2 , bw = 0.5, ax=axes[0], label='Piccadilly')
sb.kdeplot(df.Volume , bw = 0.5, ax=axes[1], label='Piccadilly')
sb.kdeplot(PollutantsShar.NO2 , bw = 0.5, ax=axes[0], label='Sharston')
sb.kdeplot(df2.Volume , bw = 0.5, ax=axes[1], label='Sharston')
# labels
axes[0].set_xlabel('NO2')
axes[0].legend(loc='upper right')
axes[1].legend(loc='upper right')
axes[1].set_xlabel('Traffic Volume')
axes[0].axvline(x=PollutantsPicc.NO2.median(),
            color='blue',
            ls='--', 
            lw=2.5)
axes[0].axvline(x=PollutantsShar.NO2.median(),
            color='orange',
            ls='--', 
            lw=2.5)

axes[0].axvline(x=PollutantsPicc.NO2.mean(),
            color='violet',
            ls='--', 
            lw=2.5)
axes[0].axvline(x=PollutantsShar.NO2.mean(),
            color='red',
            ls='--', 
            lw=2.5)


axes[1].axvline(x=df.Volume.median(),
            color='blue',
            ls='--', 
            lw=2.5)
axes[1].axvline(x=df2.Volume.median(),
            color='orange',
            ls='--', 
            lw=2.5)

axes[1].axvline(x=df.Volume.mean(),
            color='violet',
            ls='--', 
            lw=2.5)
axes[1].axvline(x=df2.Volume.mean(),
            color='red',
            ls='--', 
            lw=2.5)
plt.subplots_adjust(hspace=0.3)
plt.show()

In [ ]:
print(PollutantsPicc.NO2.median())
print(PollutantsPicc.NO2.mean())
print(df.Volume.mean())
print(df.Volume.median())

In [ ]:
print(PollutantsShar.NO2.median())
print(PollutantsShar.NO2.mean())
print(df2.Volume.mean())
print(df2.Volume.median())

In [ ]:
PollutantsShar.NO2.describe()

In [ ]:
print(PollutantsPicc.NO2.mode())
print(df.Volume.mode())

In [ ]:
print(PollutantsShar.NO2.mode())
print(df2.Volume.mode())

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10,4), sharex=True)
sns.boxplot(y=PollutantsPicc.site, x=PollutantsPicc.temp, ax=axes[0],palette="Set2")
sns.boxplot(y=PollutantsShar.site, x=PollutantsShar.temp, ax=axes[1])
axes[0].set_xlabel('')
axes[0].set_ylabel('Piccadilly')
axes[1].set_ylabel('Sharston')
# Hide X and Y axes label marks
axes[0].yaxis.set_tick_params(labelleft=False)
axes[1].yaxis.set_tick_params(labelleft=False)

# Hide X and Y axes tick marks
axes[0].set_yticks([])
axes[1].set_yticks([])
axes[1].set_xlabel('Temperature (Celcius)')
axes[0].label_outer()
axes[1].label_outer()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10,4), sharex=True)
sns.boxplot(y=PollutantsPicc.site, x=PollutantsPicc.ws, ax=axes[0],palette="Set2")
sns.boxplot(y=PollutantsShar.site, x=PollutantsShar.ws, ax=axes[1])
axes[0].set_xlabel('')
axes[1].set_xlabel('wind speed (m/s)')
axes[0].set_ylabel('Piccadilly')
axes[1].set_ylabel('Sharston')
# Hide X and Y axes label marks
axes[0].yaxis.set_tick_params(labelleft=False)
axes[1].yaxis.set_tick_params(labelleft=False)

# Hide X and Y axes tick marks
axes[0].set_yticks([])
axes[1].set_yticks([])
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(10,8), sharex=True)
sns.boxplot(y=PollutantsPicc.site, x=PollutantsPicc.NO2, ax=axes[0],palette="Set2")
sns.boxplot(y=PollutantsShar.site, x=PollutantsShar.NO2, ax=axes[1])
axes[0].set_xlabel('')
axes[0].set_ylabel('Piccadilly')
axes[1].set_ylabel('Sharston')
# Hide X and Y axes label marks
axes[0].yaxis.set_tick_params(labelleft=False)
axes[1].yaxis.set_tick_params(labelleft=False)

# Hide X and Y axes tick marks
axes[0].set_yticks([])
axes[1].set_yticks([])
plt.show()

In [ ]:
fig, axes = plt.subplots(7, 2, figsize=(15,21))

sns.lineplot(x='temp', y='NO2', data=PollutantsPicc, ax=axes[0,0], label='Piccadilly')
sns.lineplot(x='temp', y='NO2', data=PollutantsShar, ax=axes[0,0], label='Sharston')

sns.lineplot(x='ws', y='NO2', data=PollutantsPicc, ax=axes[1,0], label='Piccadilly')
sns.lineplot(x='ws', y='NO2', data=PollutantsShar, ax=axes[1,0], label='Sharston')

sns.lineplot(x='date', y='NO2', data=PollutantsPicc20mn6dy4, ax=axes[2,0], label='Piccadilly')
sns.lineplot(x='date', y='NO2', data=PollutantsShar20mn6dy4, ax=axes[2,0], label='Sharston')

sns.lineplot(x='Year', y='NO2', data=PollutantsPicc, ax=axes[3,0], label='Piccadilly')
sns.lineplot(x='Year', y='NO2', data=PollutantsShar, ax=axes[3,0], label='Sharston')

sns.lineplot(x='date', y='NO2', data=PollutantsPicc20mn6, ax=axes[4,0], label='Piccadilly')
sns.lineplot(x='date', y='NO2', data=PollutantsShar20mn6, ax=axes[4,0], label='Sharston')

sns.lineplot(x='date', y='NO2', data=PollutantsPicc20mn6w23, ax=axes[5,0], label='Piccadilly')
sns.lineplot(x='date', y='NO2', data=PollutantsShar20mn6w23, ax=axes[5,0], label='Sharston')

sns.lineplot(x='date', y='NO2_moving_avg4', data=PollutantsPicc20, ax=axes[6,0], label='Piccadilly')
sns.lineplot(x='date', y='NO2_moving_avg4', data=PollutantsShar20, ax=axes[6,0], label='Sharston')


sns.lineplot(x='temp', y='Volume', data=df, ax=axes[0,1], label='Piccadilly')
sns.lineplot(x='temp', y='Volume', data=df2, ax=axes[0,1], label='Sharston')

sns.lineplot(x='ws', y='Volume', data=df, ax=axes[1,1], label='Piccadilly')
sns.lineplot(x='ws', y='Volume', data=df2, ax=axes[1,1], label='Sharston')

sns.lineplot(x='date', y='Volume', data=df20mn6dy4, ax=axes[2,1], label='Piccadilly')
sns.lineplot(x='date', y='Volume', data=df220mn6dy4, ax=axes[2,1], label='Sharston')

sns.lineplot(x='Year', y='Volume', data=df, ax=axes[3,1], label='Piccadilly')
sns.lineplot(x='Year', y='Volume', data=df2, ax=axes[3,1], label='Sharston')

sns.lineplot(x='date', y='Volume', data=df20mn6, ax=axes[4,1], label='Piccadilly')
sns.lineplot(x='date', y='Volume', data=df220mn6, ax=axes[4,1], label='Sharston')

sns.lineplot(x='date', y='Volume', data=df20mn6w23, ax=axes[5,1], label='Piccadilly')
sns.lineplot(x='date', y='Volume', data=df220mn6w23, ax=axes[5,1], label='Sharston')

sns.lineplot(x='date', y=df.Volume.rolling(window=100).mean(), data=df20, ax=axes[6,1], label='Piccadilly')
sns.lineplot(x='date', y=df2.Volume.rolling(window=100).mean(), data=df220, ax=axes[6,1], label='Sharston')


axes[4,0].xaxis.set_major_locator(plt.MaxNLocator(5))
axes[4,1].xaxis.set_major_locator(plt.MaxNLocator(5))

date_form = DateFormatter("%H")
axes[2,0].xaxis.set_major_formatter(date_form)
axes[2,1].xaxis.set_major_formatter(date_form)
date_form = DateFormatter("%d")
axes[5,0].xaxis.set_major_formatter(date_form)
axes[5,1].xaxis.set_major_formatter(date_form)


axes[0,0].legend(loc='upper right')
axes[1,0].legend(loc='upper right')
axes[2,0].legend(loc='upper right')
axes[3,0].legend(loc='upper right')
axes[4,0].legend(loc='upper right')
axes[5,0].legend(loc='upper right')
axes[6,0].legend(loc='upper right')

axes[0,1].legend(loc='upper right')
axes[1,1].legend(loc='upper right')
axes[2,1].legend(loc='upper right')
axes[3,1].legend(loc='upper right')
axes[4,1].legend(loc='upper right')
axes[5,1].legend(loc='upper right')
axes[6,1].legend(loc='upper right')

axes[0,0].set_xlabel('Temperature (Celcius)')
axes[0,1].set_xlabel('Temperature (Celcius)')

axes[1,0].set_xlabel('Wind Speed (m/s)')
axes[1,1].set_xlabel('Wind Speed (m/s)')

axes[2,0].set_xlabel('Hour')
axes[2,0].set_title('Hourly NO2 concentrations on 04/06/2020',loc = 'center')
axes[2,1].set_xlabel('Hour')
axes[2,1].set_title('Hourly Traffic Volume on 04/06/2020',loc = 'center')
axes[2,1].set_ylabel('Traffic Volume')

axes[3,0].set_title('Yearly Aggregate NO2 concentrations',loc = 'center')
axes[3,1].set_title('Yearly Aggregate Tarffic Volume',loc = 'center')
axes[3,1].set_ylabel('Traffic Volume')

axes[4,0].set_xlabel('Day')
axes[4,0].set_title('NO2 concentrations in the month of June in 2020',loc = 'center')
axes[4,1].set_xlabel('Day')
axes[4,1].set_title('Traffic Volume in the month of June in 2020',loc = 'center')
axes[4,1].set_ylabel('Traffic Volume')

axes[5,0].set_xlabel('Day of Week')
axes[5,0].set_title('NO2 concentrations on the first week of June in 2020',loc = 'center')
axes[5,1].set_xlabel('Day of Week')
axes[5,1].set_title('Traffic Volume on the first week of June in 2020',loc = 'center')
axes[5,1].set_ylabel('Traffic Volume')

axes[6,0].set_xlabel('Month')
axes[6,0].set_ylabel('Rolling Mean NO2 (100)')
axes[6,0].set_title('Rolling Mean NO2 (100) concentrations in 2020',loc = 'center')
axes[6,1].set_xlabel('Month')
axes[6,1].set_ylabel('Rolling Mean Volume (100)')
axes[6,1].set_title('Rolling Mean Volume (100) in 2020',loc = 'center')
axes[6,1].set_ylabel('Traffic Volume')

plt.subplots_adjust(wspace=0.2, hspace=0.6)
plt.show()
#rolling means for better look?

In [ ]:
fig, axes = plt.subplots(7, 2, figsize=(15,21))
#fig.suptitle('NO2                             Volume')

#First Column: NO2 ----

#sns.lineplot(x='temp', y='NO2', data=PollutantsPicc, ax=axes[0,0], label='Piccadilly')
#sns.lineplot(x='temp', y='NO2', data=PollutantsShar, ax=axes[0,0], label='Sharston')

#sns.lineplot(x='ws', y='NO2', data=PollutantsPicc, ax=axes[1,0], label='Piccadilly')
#sns.lineplot(x='ws', y='NO2', data=PollutantsShar, ax=axes[1,0], label='Sharston')

sns.lineplot(x='Hour', y='NO2', data=PollutantsPicc, ax=axes[2,0], label='Piccadilly')
sns.lineplot(x='Hour', y='NO2', data=PollutantsShar, ax=axes[2,0], label='Sharston')

sns.lineplot(x='Year', y='NO2', data=PollutantsPicc, ax=axes[3,0], label='Piccadilly')
sns.lineplot(x='Year', y='NO2', data=PollutantsShar, ax=axes[3,0], label='Sharston')

#sns.lineplot(x='Day', y='NO2', data=PollutantsPicc, ax=axes[4,0], label='Piccadilly')
#sns.lineplot(x='Day', y='NO2', data=PollutantsShar, ax=axes[4,0], label='Sharston')

sns.lineplot(x='DayofWeek', y='NO2', data=PollutantsPicc, ax=axes[4,0], label='Piccadilly')
sns.lineplot(x='DayofWeek', y='NO2', data=PollutantsShar, ax=axes[4,0], label='Sharston')

sns.lineplot(x='Month', y='NO2', data=PollutantsPicc, ax=axes[5,0], label='Piccadilly')
sns.lineplot(x='Month', y='NO2', data=PollutantsShar, ax=axes[5,0], label='Sharston')


#Second Column: Volume ----

#sns.lineplot(x='temp', y='Volume', data=df, ax=axes[0,1], label='Piccadilly')
#sns.lineplot(x='temp', y='Volume', data=df2, ax=axes[0,1], label='Sharston')

#sns.lineplot(x='ws', y='Volume', data=df, ax=axes[1,1], label='Piccadilly')
#sns.lineplot(x='ws', y='Volume', data=df2, ax=axes[1,1], label='Sharston')

sns.lineplot(x='Hour', y='Volume', data=df, ax=axes[2,1], label='Piccadilly')
sns.lineplot(x='Hour', y='Volume', data=df2, ax=axes[2,1], label='Sharston')

sns.lineplot(x='Year', y='Volume', data=df, ax=axes[3,1], label='Piccadilly')
sns.lineplot(x='Year', y='Volume', data=df2, ax=axes[3,1], label='Sharston')

#sns.lineplot(x='Day', y='Volume', data=df, ax=axes[4,1], label='Piccadilly')
#sns.lineplot(x='Day', y='Volume', data=df2, ax=axes[4,1], label='Sharston')

sns.lineplot(x='DayofWeek', y='Volume', data=df, ax=axes[4,1], label='Piccadilly')
sns.lineplot(x='DayofWeek', y='Volume', data=df2, ax=axes[4,1], label='Sharston')

sns.lineplot(x='Month', y='Volume', data=df, ax=axes[5,1], label='Piccadilly')
sns.lineplot(x='Month', y='Volume', data=df2, ax=axes[5,1], label='Sharston')

#Axes configuration ----

#date_form = DateFormatter("%m-%Y")
#axes[0,0].xaxis.set_major_locator(MonthLocator(bymonth=(1,7)))
#axes[0,0].xaxis.set_major_locator(plt.MaxNLocator(6))
#axes[0,0].xaxis.set_major_formatter(date_form)
#axes[0,1].xaxis.set_major_locator(MonthLocator(bymonth=(1,7)))
#axes[0,1].xaxis.set_major_locator(plt.MaxNLocator(6))
#axes[0,1].xaxis.set_major_formatter(date_form)
#axes[4,0].xaxis.set_major_locator(plt.MaxNLocator(5))
#axes[4,1].xaxis.set_major_locator(plt.MaxNLocator(5))

#date_form = DateFormatter("%H")
#axes[2,0].xaxis.set_major_formatter(date_form)
#axes[2,1].xaxis.set_major_formatter(date_form)
#ate_form = DateFormatter("%d")
#axes[5,0].xaxis.set_major_formatter(date_form)
#axes[5,1].xaxis.set_major_formatter(date_form)
# labels
#axes[0,0].set_xlim(['07-2016','07-2017'])

axes[0,0].legend(loc='upper right')
axes[1,0].legend(loc='upper right')
axes[2,0].legend(loc='upper right')
axes[3,0].legend(loc='upper right')
axes[4,0].legend(loc='upper right')
axes[5,0].legend(loc='upper right')
axes[6,0].legend(loc='upper right')

axes[0,1].legend(loc='upper right')
axes[1,1].legend(loc='upper right')
axes[2,1].legend(loc='upper right')
axes[3,1].legend(loc='upper right')
axes[4,1].legend(loc='upper right')
axes[5,1].legend(loc='upper right')
axes[6,1].legend(loc='upper right')

axes[0,0].set_xlabel('Temperature (Celcius)')
axes[0,1].set_xlabel('Temperature (Celcius)')

axes[1,0].set_xlabel('Wind Speed (m/s)')
axes[1,1].set_xlabel('Wind Speed (m/s)')

axes[2,0].set_xlabel('Hour')
axes[2,0].set_title('Hourly NO2 concentrations',loc = 'center')
axes[2,1].set_xlabel('Hour')
axes[2,1].set_title('Hourly Traffic Volume',loc = 'center')
axes[2,1].set_ylabel('Traffic Volume')

axes[3,0].set_title('Yearly NO2 concentrations',loc = 'center')
axes[3,1].set_title('Yearly Tarffic Volume',loc = 'center')
axes[3,1].set_ylabel('Traffic Volume')
axes[3,1].set_xticks([2016,2017,2018,2019,2020,2021])

#axes[4,0].set_xlabel('Day')
#axes[4,0].set_title('NO2 concentrations Daily throughout a month',loc = 'center')
#axes[4,1].set_xlabel('Day')
#axes[4,1].set_title('Traffic Volume Daily throughout a month',loc = 'center')
#axes[4,1].set_ylabel('Traffic Volume')

axes[4,0].set_xlabel('Day of Week')
axes[4,0].set_title('NO2 concentrations Daily throughout a week',loc = 'center')
axes[4,0].set_xticks([0,1,2,3,4,5,6])
axes[4,0].set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
axes[4,1].set_xlabel('Day of Week')
axes[4,1].set_title('Traffic Volume Daily throughout a week',loc = 'center')
axes[4,1].set_ylabel('Traffic Volume')
axes[4,1].set_xticks([0,1,2,3,4,5,6])
axes[4,1].set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])

axes[5,0].set_xlabel('Month')
axes[5,0].set_title('Monthly NO2 concentrations',loc = 'center')
axes[5,0].set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
axes[5,0].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
axes[5,1].set_xlabel('Month')
axes[5,1].set_title('Monthly Traffic Volume',loc = 'center')
axes[5,1].set_ylabel('Traffic Volume')
axes[5,1].set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
axes[5,1].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])


#axes[0,0].set_yscale('log')
#axes[0,1].set_yscale('log')

plt.subplots_adjust(wspace=0.2, hspace=0.6)
plt.show()
#rolling means for better look?

In [ ]:
fig, axes = plt.subplots(7, 2, figsize=(15,21))
#fig.suptitle('NO2                             Volume')

#First Column: NO2 ----

#sns.lineplot(x='temp', y='NO2', data=PollutantsPicc, ax=axes[0,0], label='Picaddily')
#sns.lineplot(x='temp', y='NO2', data=PollutantsShar, ax=axes[0,0], label='Sharston')

#sns.lineplot(x='ws', y='NO2', data=PollutantsPicc, ax=axes[1,0], label='Picaddily')
#sns.lineplot(x='ws', y='NO2', data=PollutantsShar, ax=axes[1,0], label='Sharston')

sns.boxplot(x='Hour', y='NO2', data=PollutantsPicc, ax=axes[2,0])

sns.boxplot(x='Year', y='NO2', data=PollutantsPicc, ax=axes[3,0])

sns.boxplot(x='Day', y='NO2', data=PollutantsPicc, ax=axes[4,0])

sns.boxplot(x='DayofWeek', y='NO2', data=PollutantsPicc, ax=axes[5,0])

sns.boxplot(x='Month', y='NO2', data=PollutantsPicc, ax=axes[6,0])


#Second Column: Volume ----

#sns.lineplot(x='temp', y='Volume', data=df, ax=axes[0,1], label='Picaddily')
#sns.lineplot(x='temp', y='Volume', data=df2, ax=axes[0,1], label='Sharston')

#sns.lineplot(x='ws', y='Volume', data=df, ax=axes[1,1], label='Picaddily')
#sns.lineplot(x='ws', y='Volume', data=df2, ax=axes[1,1], label='Sharston')

sns.boxplot(x='Hour', y='Volume', data=df, ax=axes[2,1])

sns.boxplot(x='Year', y='Volume', data=df, ax=axes[3,1])

sns.boxplot(x='Day', y='Volume', data=df, ax=axes[4,1])

sns.boxplot(x='DayofWeek', y='Volume', data=df, ax=axes[5,1])

sns.boxplot(x='Month', y='Volume', data=df, ax=axes[6,1])

#Axes configuration ----

#date_form = DateFormatter("%m-%Y")
#axes[0,0].xaxis.set_major_locator(MonthLocator(bymonth=(1,7)))
#axes[0,0].xaxis.set_major_locator(plt.MaxNLocator(6))
#axes[0,0].xaxis.set_major_formatter(date_form)
#axes[0,1].xaxis.set_major_locator(MonthLocator(bymonth=(1,7)))
#axes[0,1].xaxis.set_major_locator(plt.MaxNLocator(6))
#axes[0,1].xaxis.set_major_formatter(date_form)
#axes[4,0].xaxis.set_major_locator(plt.MaxNLocator(5))
#axes[4,1].xaxis.set_major_locator(plt.MaxNLocator(5))

#date_form = DateFormatter("%H")
#axes[2,0].xaxis.set_major_formatter(date_form)
#axes[2,1].xaxis.set_major_formatter(date_form)
#ate_form = DateFormatter("%d")
#axes[5,0].xaxis.set_major_formatter(date_form)
#axes[5,1].xaxis.set_major_formatter(date_form)
# labels
#axes[0,0].set_xlim(['07-2016','07-2017'])

axes[0,0].legend(loc='upper right')
axes[1,0].legend(loc='upper right')
axes[2,0].legend(loc='upper right')
axes[3,0].legend(loc='upper right')
axes[4,0].legend(loc='upper right')
axes[5,0].legend(loc='upper right')
axes[6,0].legend(loc='upper right')

axes[0,1].legend(loc='upper right')
axes[1,1].legend(loc='upper right')
axes[2,1].legend(loc='upper right')
axes[3,1].legend(loc='upper right')
axes[4,1].legend(loc='upper right')
axes[5,1].legend(loc='upper right')
axes[6,1].legend(loc='upper right')

axes[0,0].set_xlabel('Temperature (Celcius)')
axes[0,1].set_xlabel('Temperature (Celcius)')

axes[1,0].set_xlabel('Wind Speed (m/s)')
axes[1,1].set_xlabel('Wind Speed (m/s)')

axes[2,0].set_xlabel('Hour')
axes[2,0].set_title('Hourly NO2 concentrations (Piccadilly)',loc = 'center')
axes[2,1].set_xlabel('Hour')
axes[2,1].set_title('Hourly Traffic Volume (Piccadilly)',loc = 'center')
axes[2,1].set_ylabel('Traffic Volume')

axes[3,0].set_title('Yearly NO2 concentrations (Piccadilly)',loc = 'center')
axes[3,1].set_title('Yearly Tarffic Volume (Piccadilly)',loc = 'center')
axes[3,1].set_ylabel('Traffic Volume')

axes[4,0].set_xlabel('Day')
axes[4,0].set_title('NO2 concentrations Daily throughout a month (Piccadilly)',loc = 'center')
axes[4,1].set_xlabel('Day')
axes[4,1].set_title('Traffic Volume Daily throughout a month (Piccadilly)',loc = 'center')
axes[4,1].set_ylabel('Traffic Volume')

axes[5,0].set_xlabel('Day of Week')
axes[5,0].set_title('NO2 concentrations Daily throughout a week (Piccadilly)',loc = 'center')
axes[5,0].set_xticks([0,1,2,3,4,5,6])
axes[5,0].set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
axes[5,1].set_xlabel('Day of Week')
axes[5,1].set_title('Traffic Volume Daily throughout a week (Piccadilly)',loc = 'center')
axes[5,1].set_ylabel('Traffic Volume')
axes[5,1].set_xticks([0,1,2,3,4,5,6])
axes[5,1].set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])

axes[6,0].set_xlabel('Month')
axes[6,0].set_title('Monthly NO2 concentrations (Piccadilly)',loc = 'center')
axes[6,0].set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
axes[6,0].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
axes[6,1].set_xlabel('Month')
axes[6,1].set_title('Monthly Traffic Volume (Piccadilly)',loc = 'center')
axes[6,1].set_ylabel('Traffic Volume')
axes[6,1].set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
axes[6,1].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])


#axes[0,0].set_yscale('log')
#axes[0,1].set_yscale('log')

plt.subplots_adjust(wspace=0.2, hspace=0.6)
plt.show()
#rolling means for better look?

In [ ]:
fig, axes = plt.subplots(7, 2, figsize=(15,21))
#fig.suptitle('NO2                             Volume')

#First Column: NO2 ----

#sns.lineplot(x='temp', y='NO2', data=PollutantsPicc, ax=axes[0,0], label='Picaddily')
#sns.lineplot(x='temp', y='NO2', data=PollutantsShar, ax=axes[0,0], label='Sharston')

#sns.lineplot(x='ws', y='NO2', data=PollutantsPicc, ax=axes[1,0], label='Picaddily')
#sns.lineplot(x='ws', y='NO2', data=PollutantsShar, ax=axes[1,0], label='Sharston')

sns.boxplot(x='Hour', y='NO2', data=PollutantsShar, ax=axes[2,0])

sns.boxplot(x='Year', y='NO2', data=PollutantsShar, ax=axes[3,0])

sns.boxplot(x='Day', y='NO2', data=PollutantsShar, ax=axes[4,0])

sns.boxplot(x='DayofWeek', y='NO2', data=PollutantsShar, ax=axes[5,0])

sns.boxplot(x='Month', y='NO2', data=PollutantsShar, ax=axes[6,0])


#Second Column: Volume ----

#sns.lineplot(x='temp', y='Volume', data=df, ax=axes[0,1], label='Picaddily')
#sns.lineplot(x='temp', y='Volume', data=df2, ax=axes[0,1], label='Sharston')

#sns.lineplot(x='ws', y='Volume', data=df, ax=axes[1,1], label='Picaddily')
#sns.lineplot(x='ws', y='Volume', data=df2, ax=axes[1,1], label='Sharston')

sns.boxplot(x='Hour', y='Volume', data=df2, ax=axes[2,1])

sns.boxplot(x='Year', y='Volume', data=df2, ax=axes[3,1])

sns.boxplot(x='Day', y='Volume', data=df2, ax=axes[4,1])

sns.boxplot(x='DayofWeek', y='Volume', data=df2, ax=axes[5,1])

sns.boxplot(x='Month', y='Volume', data=df2, ax=axes[6,1])

#Axes configuration ----

#date_form = DateFormatter("%m-%Y")
#axes[0,0].xaxis.set_major_locator(MonthLocator(bymonth=(1,7)))
#axes[0,0].xaxis.set_major_locator(plt.MaxNLocator(6))
#axes[0,0].xaxis.set_major_formatter(date_form)
#axes[0,1].xaxis.set_major_locator(MonthLocator(bymonth=(1,7)))
#axes[0,1].xaxis.set_major_locator(plt.MaxNLocator(6))
#axes[0,1].xaxis.set_major_formatter(date_form)
#axes[4,0].xaxis.set_major_locator(plt.MaxNLocator(5))
#axes[4,1].xaxis.set_major_locator(plt.MaxNLocator(5))

#date_form = DateFormatter("%H")
#axes[2,0].xaxis.set_major_formatter(date_form)
#axes[2,1].xaxis.set_major_formatter(date_form)
#ate_form = DateFormatter("%d")
#axes[5,0].xaxis.set_major_formatter(date_form)
#axes[5,1].xaxis.set_major_formatter(date_form)
# labels
#axes[0,0].set_xlim(['07-2016','07-2017'])

axes[0,0].legend(loc='upper right')
axes[1,0].legend(loc='upper right')
axes[2,0].legend(loc='upper right')
axes[3,0].legend(loc='upper right')
axes[4,0].legend(loc='upper right')
axes[5,0].legend(loc='upper right')
axes[6,0].legend(loc='upper right')

axes[0,1].legend(loc='upper right')
axes[1,1].legend(loc='upper right')
axes[2,1].legend(loc='upper right')
axes[3,1].legend(loc='upper right')
axes[4,1].legend(loc='upper right')
axes[5,1].legend(loc='upper right')
axes[6,1].legend(loc='upper right')

axes[0,0].set_xlabel('Temperature (Celcius)')
axes[0,1].set_xlabel('Temperature (Celcius)')

axes[1,0].set_xlabel('Wind Speed (m/s)')
axes[1,1].set_xlabel('Wind Speed (m/s)')

axes[2,0].set_xlabel('Hour')
axes[2,0].set_title('Hourly NO2 concentrations (Sharston)',loc = 'center')
axes[2,1].set_xlabel('Hour')
axes[2,1].set_title('Hourly Traffic Volume (Sharston)',loc = 'center')
axes[2,1].set_ylabel('Traffic Volume')

axes[3,0].set_title('Yearly NO2 concentrations (Sharston)',loc = 'center')
axes[3,1].set_title('Yearly Tarffic Volume (Sharston)',loc = 'center')
axes[3,1].set_ylabel('Traffic Volume')

axes[4,0].set_xlabel('Day')
axes[4,0].set_title('NO2 concentrations Daily throughout a month (Sharston)',loc = 'center')
axes[4,1].set_xlabel('Day')
axes[4,1].set_title('Traffic Volume Daily throughout a month (Sharston)',loc = 'center')
axes[4,1].set_ylabel('Traffic Volume')

axes[5,0].set_xlabel('Day of Week')
axes[5,0].set_title('NO2 concentrations Daily throughout a week (Sharston)',loc = 'center')
axes[5,0].set_xticks([0,1,2,3,4,5,6])
axes[5,0].set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])
axes[5,1].set_xlabel('Day of Week')
axes[5,1].set_title('Traffic Volume Daily throughout a week (Sharston)',loc = 'center')
axes[5,1].set_ylabel('Traffic Volume')
axes[5,1].set_xticks([0,1,2,3,4,5,6])
axes[5,1].set_xticklabels(['Mon','Tue','Wed','Thu','Fri','Sat','Sun'])

axes[6,0].set_xlabel('Month')
axes[6,0].set_title('Monthly NO2 concentrations (Sharston)',loc = 'center')
axes[6,0].set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
axes[6,0].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
axes[6,1].set_xlabel('Month')
axes[6,1].set_title('Monthly Traffic Volume (Sharston)',loc = 'center')
axes[6,1].set_ylabel('Traffic Volume')
axes[6,1].set_xticks([1,2,3,4,5,6,7,8,9,10,11,12])
axes[6,1].set_xticklabels(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])



#axes[0,0].set_yscale('log')
#axes[0,1].set_yscale('log')

plt.subplots_adjust(wspace=0.2, hspace=0.6)
plt.show()
#rolling means for better look?

In [ ]:
from statsmodels.graphics import tsaplots

In [ ]:
#plot autocorrelation function
fig= plt.figure(figsize=(10,5))
tsaplots.plot_acf(PollutantsPicc.NO2, lags=60, color='g',  title='Autocorrelation function (Piccadilly)')
plt.xlabel('lags')
plt.show()

In [ ]:
#plot autocorrelation function
fig= plt.figure(figsize=(10,5))
tsaplots.plot_acf(PollutantsPicc.NO2, lags=4000, color='g',  title='Autocorrelation function (Piccadilly)')
plt.xlabel('lags')
plt.show()

In [ ]:
#plot autocorrelation function
fig= plt.figure(figsize=(10,5))
tsaplots.plot_acf(PollutantsPicc.NO2, lags=40000, color='g', title='Autocorrelation function (Piccadilly)')
plt.xlabel('lags')
plt.show()

In [ ]:
#plot autocorrelation function
fig= plt.figure(figsize=(10,5))
tsaplots.plot_acf(PollutantsShar.NO2, lags=60, color='g', title='Autocorrelation function (Sharston)')
plt.xlabel('lags')
plt.show()

In [ ]:
#plot autocorrelation function
fig= plt.figure(figsize=(10,5))
tsaplots.plot_acf(PollutantsShar.NO2, lags=4000, color='g', title='Autocorrelation function (Sharston)')
plt.xlabel('lags')
plt.show()

In [ ]:
#plot autocorrelation function
fig= plt.figure(figsize=(10,5))
tsaplots.plot_acf(PollutantsShar.NO2, lags=40000, color='g', title='Autocorrelation function (Sharston)')
plt.xlabel('lags')
plt.show()

In [ ]:
fig= plt.figure(figsize=(12,8))
sns.boxplot(PollutantsPicc.wd, PollutantsPicc.NO2)
plt.xlabel('Wind Direction')
plt.show()

In [ ]:
fig= plt.figure(figsize=(12,8))
sns.boxplot(PollutantsShar.wd, PollutantsShar.NO2)
plt.xlabel('Wind Direction')
plt.show()

# **Piccadilly site Modeling**

## **Prepare Data Where X contains the predictor variables and y the target**

### Varcomb1

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsPicc.iloc[:, [5,6,7]].values
y = PollutantsPicc.iloc[:,3].values

In [ ]:
#onehotencoded
X = PollutantsPicc.iloc[:, [6,7,14,15,16,17,18,19,20,21]].values
y = PollutantsPicc.iloc[:,3].values

### Varcomb2

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsPicc.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = PollutantsPicc.iloc[:,3].values

In [ ]:
#onehotencoded
X = PollutantsPicc.iloc[:, [6,7,16,17,19,20,22,23,24,26,27,28,34,35,36,37,38,39,40,41]].values
y = PollutantsPicc.iloc[:,3].values

### Varcomb3

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = df.iloc[:, [6,9,10,11,14,15,16,17,18,19,20]].values
y = df.iloc[:,8].values

In [ ]:
#onehotencoded
X = df.iloc[:, [6,10,11,14,15,16,17,18,19,20,26,27,
                28,29,30,31,32,33,34,35,36,37,38,39,40,41]].values
y = df.iloc[:,8].values


### Varcomb4

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsPicc.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = PollutantsPicc.iloc[:,22].values

In [ ]:
#onehotencoded
X = PollutantsPicc.iloc[:, [6,7,16,17,19,20,21,22,23,24,25,26,27,28,34,35,36,37,38,39,40,41]].values
y = PollutantsPicc.iloc[:,29].values

### Varcomb5

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsPicc.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = PollutantsPicc.iloc[:,23].values

In [ ]:
#onehotencoded
X = PollutantsPicc.iloc[:, [6,7,16,17,19,20,22,23,24,26,27,28,34,35,36,37,38,39,40,41]].values
y = PollutantsPicc.iloc[:,30].values

### Varcomb6

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsPicc.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = PollutantsPicc.iloc[:,24].values

In [ ]:
#onehotencoded
X = PollutantsPicc.iloc[:, [6,7,16,17,19,20,22,23,24,26,27,28,34,35,36,37,38,39,40,41]].values
y = PollutantsPicc.iloc[:,31].values

### Varcomb7

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = df.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = df.iloc[:,23].values

In [ ]:
#onehotencoded
X = df.iloc[:, [6,7,16,17,19,20,22,23,24,26,27,28,34,35,36,37,38,39,40,41]].values
y = df.iloc[:,32].values

### Perform Train Test Split

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
print(scaler.fit(X))

StandardScaler()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state=42)

## **Random Forest**

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 10 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 10, cv = 10, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
#print the best parameters found
rf_random.best_params_

In [ ]:
# Run and fit the model
rfregressor = RandomForestRegressor(n_estimators = 200,max_depth = 10, 
                                  max_features= 'sqrt', min_samples_split = 5, min_samples_leaf= 2,
                                  bootstrap = True, random_state = 42)
rfregressor.fit(X_train,y_train)

In [ ]:
#Run and fit the model
rfregressor = RandomForestRegressor(n_estimators = 400,max_depth = 60, 
                                  max_features= 'sqrt', min_samples_split = 10, min_samples_leaf= 1,
                                  bootstrap = False, random_state = 42)
rfregressor.fit(X_train,y_train)

In [ ]:
# Generates Predictions
predictr=rfregressor.predict(X_test)
predictr

In [ ]:
# Evaluating the Algorithm
from sklearn import metrics
print(rfregressor.score(X_train, y_train))
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictr))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictr))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, 
                                                                     predictr)))
print("R2: %.4f" % r2_score(y_test, predictr))

In [ ]:
#CV score
scores = cross_val_score(rfregressor, X_train, y_train, 
                         scoring='r2', cv=10,n_jobs=-1)
print(scores)
print(scores.mean())

### RF Plots 

In [ ]:
# generat a plot of observed vs predicted values on a log scale
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictr, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(predictr), max(y_test))
p2 = min(min(predictr), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()

In [ ]:
# generat a plot of observed vs predicted values
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictr, c='crimson')

p1 = max(max(predictr), max(y_test))
p2 = min(min(predictr), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()

## **XGBoost**

In [ ]:
# Set range for Number estimators
n_estimators = [int(x) for x in np.linspace(start = 1000, stop = 10000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Learning rate investigated values
learning_rate = [0.01,0.03,0.09,0.1,0.3]
# colsample_by tree investigated values
colsample_bytree = [0.3,0.7]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'learning_rate': learning_rate,
               'colsample_bytree': colsample_bytree}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
xgbr = xgb.XGBRegressor()
# Random search of parameters, using 10 fold cross validation, 
# search across 100 different combinations, and use all available cores
xgbr_random = RandomizedSearchCV(estimator = xgbr, param_distributions = random_grid, 
                               n_iter = 10, cv = 10, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
xgbr_random.fit(X_train, y_train)

In [ ]:
#Find best parameters from random search
xgbr_random.best_params_

In [ ]:
#Run and fit the model
xgbr = xgb.XGBRegressor(n_estimators=4000, max_depth=10,
                       learning_rate=0.03,
                       colsample_bytree = 0.3,
                       random_state=42)
xgbr.fit(X_train,y_train)

In [ ]:
#Run and fit the model
xgbr = xgb.XGBRegressor(n_estimators=10000, max_depth=9,
                       learning_rate=0.03,
                       colsample_bytree = 0.7,
                       random_state=42)
xgbr.fit(X_train,y_train)

In [ ]:
# generate Predictions
predictx=xgbr.predict(X_test)
predictx

In [ ]:
# Evaluating the Algorithm
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictx))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictx))  
print('Root Mean Squared Error:', 
      np.sqrt(metrics.mean_squared_error(y_test, predictx)))
print("R2: %.4f" % r2_score(y_test, predictx))
print(xgbr.score(X_train, y_train))

In [ ]:
#CV score
scores = cross_val_score(xgbr, X_train, y_train, 
                         scoring='r2', cv=10,n_jobs=-1)
print(scores)
print(scores.mean())

### XGB Plots

In [ ]:
# generat a plot of observed vs predicted values on a log scale
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictx, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(predictx), max(y_test))
p2 = min(min(predictx), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()

In [ ]:
# generate a plot of observed vs predicted values
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictx, c='crimson')

p1 = max(max(predictx), max(y_test))
p2 = min(min(predictx), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()

# **Sharston site Modeling**

## Prepare Data Where X contains the predictor variables and y the target

### Varcomb1

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsShar.iloc[:, [5,6,7]].values
y = PollutantsShar.iloc[:,3].values

In [ ]:
#OHE
X = PollutantsShar.iloc[:, [6,7,26,27,28,29,30,31,32]].values
y = PollutantsShar.iloc[:,3].values

### Varcomb2

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsShar.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = PollutantsShar.iloc[:,3].values

In [ ]:
#OHE
X = PollutantsShar.iloc[:, [6,7,14,15,16,17,18,19,20,26,27,28,29,30,31,32,33,34,35,36,37,38,39]].values
y = PollutantsShar.iloc[:,3].values

### Varcomb3

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = df2.iloc[:, [6,9,10,11,14,15,16,17,18,19,20]].values
y = df2.iloc[:,8].values

In [ ]:
#OHE
X = df2.iloc[:, [6,10,11,14,15,16,17,18,19,20,21,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40]].values
y = df2.iloc[:,8].values

### Varcomb4

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsShar.iloc[:, [5,6,7,14,15,16,17,18,19,20,21]].values
y = PollutantsShar.iloc[:,22].values

In [ ]:
#OHE
X = PollutantsShar.iloc[:, [6,7,14,15,16,17,18,19,20,21,26,27,28,29,30,31,32,33,34,35,36,37,38,39]].values
y = PollutantsShar.iloc[:,22].values

### Varcomb5

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsShar.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = PollutantsShar.iloc[:,23].values
#dependant variable is now the rolling mean(25)

0.9890059805115405

In [ ]:
#OHE
X = PollutantsShar.iloc[:, [6,7,14,15,16,17,18,19,20,21,26,27,28,29,30,31,32,33,34,35,36,37,38,39]].values
y = PollutantsShar.iloc[:,23].values

### Varcomb6

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = PollutantsShar.iloc[:, [5,6,7,14,15,16,17,18,19,20]].values
y = PollutantsShar.iloc[:,24].values

In [ ]:
#OHE
X = PollutantsShar.iloc[:, [6,7,14,15,16,17,18,19,20,21,26,27,28,29,30,31,32,33,34,35,36,37,38,39]].values
y = PollutantsShar.iloc[:,24].values

### Varcomb7

In [ ]:
#variable numbering depends on order of feature engineering and merging
X = df2.iloc[:, [6,9,10,11,14,15,16,17,18,19,20]].values
y = df2.iloc[:,23].values

In [ ]:
#OHE
X = PollutantsShar.iloc[:, [6,10,11,14,15,16,17,18,19,20,21,26,27,28,29,30,31,32,33,34,35,36,37,38,39]].values
y = PollutantsShar.iloc[:,23].values

### Perform Train test split

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
print(scaler.fit(X))

StandardScaler()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state=42)

## Random Forest

In [ ]:
#Grid for Random Search
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 10 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 10, cv = 10, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
#find best parameters
rf_random.best_params_

In [ ]:
#Run and fit the model fro Varcomb1
rfregressor = RandomForestRegressor(n_estimators = 200,max_depth = 10, 
                                  max_features= 'sqrt', min_samples_split = 5, min_samples_leaf= 2,
                                  bootstrap = True, random_state = 42)
rfregressor.fit(X_train,y_train)

In [ ]:
#Run and fit the model for Varcomb2
rfregressor = RandomForestRegressor(n_estimators = 400,max_depth = 60, 
                                  max_features= 'sqrt', min_samples_split = 10, min_samples_leaf= 1,
                                  bootstrap = False, random_state = 42)
rfregressor.fit(X_train,y_train)

In [ ]:
# Prediction
predictr=rfregressor.predict(X_test)
predictr

In [ ]:
#Evaluation
print("MSE: %.4f" % mean_squared_error(y_test, predictr))
print("RMSE: %.4f" % math.sqrt(mean_squared_error(y_test, predictr)))
print("MAE: %.4f" % mean_absolute_error(y_test, predictr))
print("R2: %.4f" % r2_score(y_test, predictr))
print(rfregressor.score(X_train, y_train))

In [ ]:
#CV score
from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfregressor, X_train, y_train, 
                         scoring='r2', cv=10,n_jobs=-1)
print(scores)
print(scores.mean())

### RF plots

In [ ]:
#Scatter plot to show predicted and Observed values
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictr, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(predictr), max(y_test))
p2 = min(min(predictr), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()

In [ ]:
#vcomb1
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictr, c='crimson')


p1 = max(max(predictr), max(y_test))
p2 = min(min(predictr), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()

## XGBoost

In [ ]:
#Grid for Random Search
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 2000, stop = 20000, num = 10)]
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# 
learning_rate = [0.01,0.03,0.09,0.1,0.3]
# 
colsample_bytree = [0.3,0.7]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'learning_rate': learning_rate,
               'colsample_bytree': colsample_bytree}
print(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
xgb = xgb.XGBRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = random_grid, 
                               n_iter = 10, cv = 5, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
xgb_random.fit(X_train, y_train)

In [ ]:
xgb_random.best_params_

In [ ]:
#vcomb1
xgbr = xgb.XGBRegressor(n_estimators=4000, max_depth=10,
                       learning_rate=0.03,
                       colsample_bytree = 0.3,
                       random_state=42)
xgbr.fit(X_train,y_train)

In [ ]:
xgbr = xgb.XGBRegressor(n_estimators=10000, max_depth=9,
                       learning_rate=0.03,
                       colsample_bytree = 0.7,
                       random_state=42)
xgbr.fit(X_train,y_train)

In [ ]:
# Prediction
predictx=xgbr.predict(X_test)
predictx

In [ ]:
print("MSE: %.4f" % mean_squared_error(y_test, predictx))
print("RMSE: %.4f" % math.sqrt(mean_squared_error(y_test, predictx)))
print("MAE: %.4f" % mean_absolute_error(y_test, predictx))
print("R2: %.4f" % r2_score(y_test, predictx))
#print(xgbr.score(X_train, y_train))

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(xgbr, X_train, y_train, 
                         scoring='r2', cv=10,n_jobs=-1)
print(scores)
print(scores.mean())

In [ ]:
Xdf = pd.DataFrame(X)
Xf = list(Xdf.columns)
Xdf = pd.DataFrame(X)
Xf = list(Xdf.columns)
# Get numerical feature importances
importances = list(xgbr.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(Xf, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

### xgb plots

In [ ]:
#Scatter plot to show predicted and Observed values
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictx, c='crimson')

plt.xscale('log')
plt.yscale('log')

p1 = max(max(predictx), max(y_test))
p2 = min(min(predictx), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()

In [ ]:
#vcomb
plt.figure(figsize=(8,8))
plt.scatter(y_test, predictx, c='crimson')


p1 = max(max(predictx), max(y_test))
p2 = min(min(predictx), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('Observed', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.axis('equal')
plt.show()